# TP4

# Word2Vec

In [ ]:
import os
import sys
import re
import nltk
from nltk.tokenize import sent_tokenize

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Segmentation des corpus en phrases

In [ ]:
# Segmentation en phrases

infile = "../tp4/data/full_left.txt"
outfile = "../tp4/data/sent_left.txt"

LIMIT = None

with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content = content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")



infile = "../tp4/data/full_right.txt"
outfile = "../tp4/data/sent_right.txt"

LIMIT = None

with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content = content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")
print("Done")




### Concaténation des expressions liées au suffrage féminin (pour pouvoir effectuer les tâches de similarité et de champ sémantique, où on ne peut manipuler que des mots uniques)

In [ ]:
# Chemin vers les fichiers
source_file_path = "../tp4/data/sent_left.txt"
destination_file_path = "../tp4/data/sent_left_concat.txt"

# Lecture du fichier source et remplacement du texte
with open(source_file_path, 'r', encoding='utf-8') as source_file:
    content = source_file.read()
    replaced_content = content.replace("vote des femmes", "votedesfemmes") \
                                .replace("suffrage des femmes", "suffragedesfemmes") \
                                .replace("suffrage féminin", "suffragefeminin") \
                                .replace("vote féminin", "votefeminin") \
                                .replace("suffrage feminin", "suffragefeminin") \
                                .replace("vote feminin", "votefeminin")

# Écriture du contenu modifié dans le fichier de destination
with open(destination_file_path, 'w', encoding='utf-8') as destination_file:
    destination_file.write(replaced_content)




# Chemin vers les fichiers
source_file_path = "../tp4/data/sent_right.txt"
destination_file_path = "../tp4/data/sent_right_concat.txt"

# Lecture du fichier source et remplacement du texte
with open(source_file_path, 'r', encoding='utf-8') as source_file:
    content = source_file.read()
    replaced_content = content.replace("vote des femmes", "votedesfemmes") \
                                .replace("suffrage des femmes", "suffragedesfemmes") \
                                .replace("suffrage féminin", "suffragefeminin") \
                                .replace("vote féminin", "votefeminin") \
                                .replace("suffrage feminin", "suffragefeminin") \
                                .replace("vote feminin", "votefeminin")

# Écriture du contenu modifié dans le fichier de destination
with open(destination_file_path, 'w', encoding='utf-8') as destination_file:
    destination_file.write(replaced_content)



# Entraînment des modèles de langue

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]
        

### Modèle Droit

In [ ]:
infile = f"../tp4/data/sent_right_concat.txt"
sentences = MySentences(infile)

In [ ]:
bigram_phrases = Phrases(sentences)
bigram_phraser = Phraser(phrases_model=bigram_phrases)

trigram_phrases = Phrases(bigram_phraser[sentences])
trigram_phraser = Phraser(phrases_model=trigram_phrases)

corpus1 = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
%%time
model = Word2Vec(
    corpus1, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 10 mots avant et après le mot observé
    min_count=3, # On ignore les mots qui n'apparaissent pas au moins 3 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=7 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
outfile = f"../tp4/data/right.model"
model.save(outfile)

### Modèle Gauche

In [ ]:
infile = f"../tp4/data/sent_left_concat.txt"
sentences = MySentences(infile)

In [ ]:
bigram_phrases = Phrases(sentences)
bigram_phraser = Phraser(phrases_model=bigram_phrases)

trigram_phrases = Phrases(bigram_phraser[sentences])
trigram_phraser = Phraser(phrases_model=trigram_phrases)

corpus2 = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
%%time
model = Word2Vec(
    corpus2, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 10 mots avant et après le mot observé
    min_count=3, # On ignore les mots qui n'apparaissent pas au moins 3 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=7 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
outfile = f"../tp4/data/left.model"
model.save(outfile)

# Test des modèles de langue sur les données des corpus

### Test similarité : Modèle droit

In [ ]:
model = Word2Vec.load("../tp4/data/right.model")

pairs = [
    ("suffragefeminin", "defavorable"),
    ("suffragedesfemmes", "defavorable"),
    ("votefeminin", "defavorable"),
    ("votedesfemmes", "defavorable"),
    ("suffragefeminin", "danger"),     
    ("suffragedesfemmes", "danger"),
    ("votefeminin", "danger"),
    ("votedesfemmes", "danger"),
    ("suffragefeminin", "risque"),
    ("suffragedesfemmes", "risque"),
    ("votefeminin", "risque"),
    ("votedesfemmes", "risque"),
    ("suffragefeminin", "menace"),
    ("suffragedesfemmes", "menace"),
    ("votefeminin", "menace"),
    ("votedesfemmes", "menace"),
    ("suffragefeminin", "mauvais"),
    ("suffragedesfemmes", "mauvais"),
    ("votefeminin", "mauvais"),
    ("votedesfemmes", "mauvais"),
    ("suffragefeminin", "avancee"),
    ("suffragedesfemmes", "avancee"),
    ("votefeminin", "avancee"),
    ("votedesfemmes", "avancee"),
    ("suffragefeminin", "progres"),     
    ("suffragedesfemmes", "progres"),
    ("votefeminin", "progres"),
    ("votedesfemmes", "progres"),
    ("suffragefeminin", "revolution"),
    ("suffragedesfemmes", "revolution"),
    ("votefeminin", "revolution"),
    ("votedesfemmes", "revolution"),
    ("suffragefeminin", "juste"),
    ("suffragedesfemmes", "juste"),
    ("votefeminin", "juste"),
    ("votedesfemmes", "juste"),
    ("suffragefeminin", "equitable"),
    ("suffragedesfemmes", "equitable"),
    ("votefeminin", "equitable"),
    ("votedesfemmes", "equitable")
]

results = []
for pair in pairs:
    similarity_score = model.wv.similarity(pair[0], pair[1])
    results.append([pair[0], pair[1], similarity_score])

# Affichage des résultats
print("mot1\tmot2\tSimilarité")
for result in results:
    print(f"{result[0]}\t{result[1]}\t{result[2]}")

### Test similarité : Modèle gauche

In [ ]:
model = Word2Vec.load("../tp4/data/left.model")

pairs = [
    ("suffragefeminin", "defavorable"),
    ("suffragedesfemmes", "defavorable"),
    ("votefeminin", "defavorable"),
    ("votedesfemmes", "defavorable"),
    ("suffragefeminin", "danger"),     
    ("suffragedesfemmes", "danger"),
    ("votefeminin", "danger"),
    ("votedesfemmes", "danger"),
    ("suffragefeminin", "risque"),
    ("suffragedesfemmes", "risque"),
    ("votefeminin", "risque"),
    ("votedesfemmes", "risque"),
    ("suffragefeminin", "menace"),
    ("suffragedesfemmes", "menace"),
    ("votefeminin", "menace"),
    ("votedesfemmes", "menace"),
    ("suffragefeminin", "mauvais"),
    ("suffragedesfemmes", "mauvais"),
    ("suffragefeminin", "avancee"),
    ("suffragedesfemmes", "avancee"),
    ("votefeminin", "avancee"),
    ("votedesfemmes", "avancee"),
    ("suffragefeminin", "progres"),     
    ("suffragedesfemmes", "progres"),
    ("votefeminin", "progres"),
    ("votedesfemmes", "progres"),
    ("suffragefeminin", "revolution"),
    ("suffragedesfemmes", "revolution"),
    ("votefeminin", "revolution"),
    ("votedesfemmes", "revolution"),
    ("suffragefeminin", "juste"),
    ("suffragedesfemmes", "juste"),
    ("votefeminin", "juste"),
    ("votedesfemmes", "juste"),
    ("suffragefeminin", "equitable"),
    ("suffragedesfemmes", "equitable"),
    ("votefeminin", "equitable"),
    ("votedesfemmes", "equitable")
]

results = []
for pair in pairs:
    similarity_score = model.wv.similarity(pair[0], pair[1])
    results.append([pair[0], pair[1], similarity_score])

# Affichage des résultats
print("mot1\tmot2\tSimilarité")
for result in results:
    print(f"{result[0]}\t{result[1]}\t{result[2]}")

### Test champ lexical : Modèle droit

In [ ]:
model = Word2Vec.load("../tp4/data/right.model")

# Liste des mots à analyser
mots = ["votedesfemmes"]

# Appliquer la fonction most_similar pour chaque mot et imprimer les résultats
for mot in mots:
    resultats = model.wv.most_similar(mot, topn=30)
    print(f"Résultats pour '{mot}':")
    print(resultats)
    print()

### Test champ lexical : Modèle gauche

In [ ]:
model = Word2Vec.load("../tp4/data/left.model")

# Liste des mots à analyser
mots = ["votedesfemmes"]

# Appliquer la fonction most_similar pour chaque mot et imprimer les résultats
for mot in mots:
    resultats = model.wv.most_similar(mot, topn=30)
    print(f"Résultats pour '{mot}':")
    print(resultats)
    print()